## Inner approximation: verify whether a given behavior $\vec{p}$ is in $\mathcal{Q}_d$

## Method 2: seesaw algorithm with semidefinite programming



A quantum behavior satisfies $\vec{p} = (p_1, p_2, ..., p_n)\in\mathcal{Q}_d$ if and only if the solution of the following optimization is zero.

(Eq. 1)
$$\begin{align}
\min_{X, P}\quad & \mathrm{Tr}(XP)\\
\mathrm{s.t.}\quad & X \ge 0, ~ \mathrm{diag}(X) = 1, ~ X_{0i} = \sqrt{p_i},\\
& X_{ij} = 0 ~~ \text{for} ~~ \{i,j\}\in E,\\
& P\ge 0, ~ P^2 = P, ~ \mathrm{rank}(P) = n - d + 1.
\end{align}
$$

This can be solved by alternatingly performing the following two optimization problems

(Eq. 2)
$$\begin{align}
\min_{X}\quad & \mathrm{Tr}(XP_\star)\\
\mathrm{s.t.}\quad & X \ge 0, ~ \mathrm{diag}(X) = 1, ~ X_{0i} = \sqrt{p_i},\\
& X_{ij} = 0 ~~ \text{for} ~~ \{i,j\}\in E.
\end{align}
$$

(Eq. 3)
$$\begin{align}
\min_{P}\quad & \mathrm{Tr}(X_\star P)\\
\mathrm{s.t.}\quad 
& P\ge 0, ~ P^2 = P, ~ \mathrm{rank}(P) = n - d + 1.
\end{align}
$$

In [1]:
import numpy as np
import cvxpy as cvx
import networkx as nx
import scipy.linalg as lin
from scipy.stats import unitary_group, ortho_group

In [2]:
# Define the graph (in graph6 format)
graph = nx.from_graph6_bytes(b'H{O_okK')

# draw the graph
nx.draw_networkx(graph)


# G is the adjacency matrix
G = nx.adjacency_matrix(graph, dtype=float).todense()
#print("The adjacency matrix:\n", G)


# n is the number of vertices
n = G.shape[0]
#print("Number of vertices:", n)

/tmp/ipykernel_17249/811224743.py:9: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  G = nx.adjacency_matrix(graph, dtype=float).todense()


In [3]:
# The SDP in (Eq. 2)


GG = np.zeros((n + 1, n + 1))
GG[1:, 1:] = G

P = cvx.Parameter((n + 1, n + 1), hermitian=True)

# sqrt(p)
proot = cvx.Parameter(n, nonneg=True)

X = cvx.Variable((n + 1, n + 1), hermitian=True)


obj = cvx.Minimize(cvx.trace(cvx.real(P @ X)))
cons = [X >> 0, cvx.diag(X) == 1,
        X[0, 1:] == proot,
        cvx.multiply(GG, X) == 0,
       ]

prob = cvx.Problem(obj, cons)

In [4]:
# Implement the seesaw algorithm

# Inputs:
# d is the dimension to test
# num is the number of different initial P_\star to try (default = 5)
# tol is the tolerance for convergence testing  (default = 1e-8)

# Output: the minial \min(Tr(XP)) found


def verify(d, num=5, tol=1e-8):
    
    
    # diag of n - d + 1 projector
    D = np.diag([1] * (n - d + 1) + [0] * d)
    
    
    # initial the minimum of Tr(XP) to be infinity
    xm = np.inf
    
    for i in range(num):

        U = unitary_group.rvs(n + 1) 

        P.value = U @ D @ U.conj().T

        #print(lin.eigh(P.value, eigvals_only=True))


        err = np.inf
        val = np.inf

        while err > tol:

            try:
                prob.solve(solver="MOSEK", verbose=False)
            except:
                prob.solve(solver="SCS", eps=1e-4, verbose=False)

            if prob.value == np.inf:
                xm = np.inf
                break
            else:
                w, v = lin.eigh(X.value)

                P.value = v @ D @ v.conj().T

                err = val - prob.value
                val = prob.value

        if prob.value == np.inf:
            print("Input is not a quantum behavior for any dimension.")
            break

        if val < xm:
            xm = val
    #print(xm)
    return xm

In [5]:
p1 = np.array([1/3,1/3,1/3,1/3,1/3,1/3,1/3,1/3,1/3])
p2 = np.array([1/2,1/4,1/4,1/2,0,0,1/4,1/4,1])
p3 = np.array([5/12,7/24,7/24,5/12,1/6,1/6,7/24,7/24,2/3])

print("p3 = (p1 + p2) / 2 ?", np.allclose(p3, (p1 + p2) / 2))

p3 = (p1 + p2) / 2 ? True


In [6]:
proot.value = np.sqrt(p1)
print(verify(d=3))
proot.value = np.sqrt(p2)
print(verify(d=3))
proot.value = np.sqrt(p3)
print(verify(d=3))

2.3468098466600848e-08
1.1702427351804293e-10
0.1754654396449999


In [7]:
p4 = np.array([1/3,1/3,1/3,0,2/3,1/3,0,0,1/3])

In [8]:
proot.value = np.sqrt(p4)
print(verify(d=4))

0.5238872912953487
